#  CYBERBULLYING DETECTION IN HINDI-ENGLISH CODE-MIXED YOUTUBE COMMENTS

## July 2018 Project
### Author: Kshiti Ballal



## Prerequisites:
* Developer key
* Video id


## 1. DATA EXTRACTION

## Importing the libraries


In [1]:
# Authentication

from googleapiclient.discovery import build 
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import pafy

# File handling

import csv
import sys
import string
import re
import pandas as pd
import random
import itertools
import pickle
import numpy as np

# Text cleaning

import nltk
from textblob import TextBlob
import nltk.classify.util as util
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder as BCF
from nltk.classify import DecisionTreeClassifier
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

## Google API Authorization

In [2]:
with open("client_secret.txt", 'r', encoding="utf-8") as fp:
        client_secret=str(fp.readline())

DEVELOPER_KEY = client_secret
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
pafy.set_api_key("client_secret")

def add_data(comments):

    count = 0
    with open("COMMENTS.csv", 'w', encoding="utf-8") as fp:
        for comment in comments:
            count += 1
            print(comment, file=fp)
            sys.stdout.write('Downloaded %d comments\r' % count)
            sys.stdout.flush()
    print('\nDone!')

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

videoId = input("ID of youtube video : \n")
url = "https://www.youtube.com/watch?v=" + videoId

#Request for comm of the Video
video = pafy.new(url)

ID of youtube video : 
zT-Gd2djRV0


## Request for Comments

In [3]:
# Extracting data from youtube.commentThreads()

results = youtube.commentThreads().list(
    part="snippet",
    maxResults=100,
    videoId=videoId,
    textFormat="plainText",
).execute()

totalResults = int(results["pageInfo"]["totalResults"])
count = 0
nextPageToken = ''
comments = []
further = True
first = True
print('Downloading Youtube comments for video:', url)
while further:
    halt = False
    print('.')
    if first == False:
        try:
            results = youtube.commentThreads().list(
                part="snippet",
                maxResults=100,
                videoId=videoId,
                textFormat="plainText",
                pageToken=nextPageToken,
            ).execute()
            totalResults = int(results["pageInfo"]["totalResults"])
        except HttpError as e:
            print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))
            halt = True
            
    if halt == False:
        count += totalResults
        for item in results["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"].replace("\n", "")
            comments.append(author + "| " + text)
        if totalResults < 100:
            further = False
            first = False
        else:
            further = True
            first = False
            try:
                nextPageToken = results["nextPageToken"]
            except KeyError as e:
                print("An KeyError error occurred: %s" % (e))
                further = False
                
add_data(comments) # call the function to save comments to csv file

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
Downloaded 4494 comments
Done!


In [104]:
## Conversion of users and comments data to Dataframe

documents = pd.read_csv('COMMENTS.csv',delimiter='|',error_bad_lines=False,header=None)
documents.columns=("Users", "Comments")
documents.drop_duplicates(subset="Comments",keep='first', inplace=True)
documents.head()

b'Skipping line 634: expected 2 fields, saw 3\nSkipping line 3553: expected 2 fields, saw 3\n'


,Users,Comments
0,Niyati Khatr,If a girl doesn't regret doing porn after rea...
1,Ritu Biswas,Bloody asshole when you're sitting for an int...
2,Beautiful life,Why did the interviwer always focus on her ne...
3,Panshul Jetwani,You're fucking judgemental
4,Yusra,Sunny you Are a Really strong Woman ❤ I wish ...


## 2. EXPLORATORY DATA ANALYSIS

In [5]:
print("Total number of comments extracted:",documents["Comments"].count())

Total number of comments extracted: 4292


In [6]:
documents['word_count'] = documents['Comments'].apply(lambda x: len(str(x).split(" ")))
documents[['Comments','word_count']].sort_values("word_count",ascending=False).head()
print("Average number of words used per comment:",round(documents["word_count"].mean()))
print("Maximum number of words used per comment:",round(documents["word_count"].max()))
print("Minimum number of words used per comment:",round(documents["word_count"].min()))

Average number of words used per comment: 26
Maximum number of words used per comment: 505
Minimum number of words used per comment: 2


In [7]:
# Checking for Missing values 

documents.isnull().values.any()
# No row or comment is empty

False

## Feature engineering

In [ ]:
# Most frequent words

#toklist=[]
#for item in documents["Comments"]:
#    tokens = word_tokenize(str(item))
#    toklist=toklist+tokens
#    wordfreqdist = nltk.FreqDist(toklist)
#print(str(wordfreqdist.most_common(20))+"\n")

## Data preprocessing

In [105]:
# Removing emojis and special symbols

def emojiremoval(fp,item):
    emoji_pattern = re.compile("["
                    u"\U0001F600-\U0001F64F"  # emoticons
                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                    u"\U00002702-\U000027B0"  # other emoticons
                    u"\U0001F923" # rolf emoji
                    u"\U000024C2-\U0001F251" "]+", flags=re.UNICODE)
    fp.write("".join(emoji_pattern.sub(r'', item))+"\n") # no emoji
      
doc=documents['Comments']
with open("COMMENTSA.csv", 'w', encoding="utf-8") as fp:
    for item in doc:
        emojiremoval(fp,item)

In [106]:
documents2 = pd.read_csv('COMMENTSA.csv',delimiter='\n',header=None)
documents2.rename(columns={0: 'Comments'}, inplace=True)
documents2.head()

,Comments
0,If a girl doesn't regret doing porn after rea...
1,Bloody asshole when you're sitting for an int...
2,Why did the interviwer always focus on her ne...
3,You're fucking judgemental
4,Sunny you Are a Really strong Woman I wish Y...


In [107]:
# Data cleaning

def normalizing(fp,item):
    
            # tokenize the sentences
            tokens = word_tokenize(item)
        
             # convert to lower case
            tokens = [w.lower() for w in tokens]
          
            # remove punctuation from each word
            table = str.maketrans('', '', string.punctuation)
            tokens = [w.translate(table) for w in tokens]
           
            # remove stopwords from each sentence
            stop_words = set(stopwords.words('english'))
            tokens = [w for w in tokens if not w in stop_words]
               
            fp.write(" ".join(tokens)+"\n")

doc2 = documents2["Comments"]
with open("COMMENTSA.csv", 'w', encoding="utf-8") as fp:
          for item in doc2:
            normalizing(fp,item)        

In [108]:
documents2 = pd.read_csv('COMMENTSA.csv',delimiter='\n',header=None)
documents2.rename(columns={0: 'Comments'}, inplace=True)
documents2.head()

,Comments
0,girl nt regret porn reaching fame heights coul...
1,bloody asshole sitting interview asking questi...
2,interviwer always focus negative points
3,fucking judgemental
4,sunny really strong woman wish slapped front e...


In [109]:
# Most frequent words through term frequency score

tf1 = (documents2['Comments']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['Comments','tf']
tf1.sort_values("tf",ascending=False).head(20)

,Comments,tf
0,,14049.0
7399,sunny,1810.0
3991,interview,716.0
7886,u,697.0
6173,questions,581.0
3995,interviewer,550.0
6444,respect,546.0
5371,nt,536.0
4631,like,535.0
4579,leone,526.0


## Sentiment analysis on cleaned data

In [114]:
def sentiment(sentifile,item):
        sentiment = 0
        positive = 0
        negative = 0
        Commentsblob = TextBlob(item)
        sentiment = Commentsblob.sentiment.polarity
        sentifile.write(item + '|' + str(sentiment) + "\n")
        sys.stdout.flush()
        if sentiment >= 0:
            positive += 1
        else:
            negative += 1  
            
doc2 = documents2["Comments"]
with open('COMMENTSA1.csv', 'w',encoding="utf-8") as sentifile:
    for item in doc2:
        sentiment(sentifile,item)

In [115]:
documents3 = pd.read_csv('COMMENTSA1.csv',delimiter='|',error_bad_lines=False,header=None)
documents3.rename(columns={0: 'Comments',1:'Sentiments'}, inplace=True)
documents3.head()

,Comments,Sentiments
0,girl nt regret porn reaching fame heights coul...,0.233482
1,bloody asshole sitting interview asking questi...,-0.153571
2,interviwer always focus negative points,-0.300000
3,fucking judgemental,-0.600000
4,sunny really strong woman wish slapped front e...,-0.183333


In [116]:
print("Number of comments with less than 0.35 sentiment score: ", len(documents3.loc[documents3['Sentiments'] < 0.35]))
print("Number of comments with greater than 0.35 sentiment score: ", len(documents3.loc[documents3['Sentiments'] >= 0.35]))
neg= documents3.loc[documents3['Sentiments'] < 0.35].sort_values(by = "Sentiments")
pos=documents3.loc[documents3['Sentiments'] >= 0.35].sort_values(by = "Sentiments")

Number of comments with less than 0.35 sentiment score:  3690
Number of comments with greater than 0.35 sentiment score:  598


In [117]:
neg.head()

,Comments,Sentiments
2060,worst interviewer worst chaubey,-1.0
2091,worst interviewer dislikes proof gtfo ibn...,-1.0
3113,dear bhupender chaubey ever asked questions po...,-1.0
3117,bhupendra chaubey worst performance,-1.0
2034,man disgusting cnn shame,-1.0


In [118]:
pos.head()

,Comments,Sentiments
2877,ibn key take away reporter interviewer even cl...,0.35
399,cool leaone lluv u,0.35
4036,sunny leone sweet xoxo,0.35
3198,real man chooses honor love respect adore wo...,0.35
1395,hats sunny ya handled patientlyif bollywood a...,0.35


In [119]:
with open("positive.txt", 'w', encoding="utf-8") as fp:
    for item in pos["Comments"]:
        fp.write(item + "\n")
        
with open("negative.txt", 'w', encoding="utf-8") as fp:
    for item in neg["Comments"]:
        fp.write(item + "\n")

## 3. Categorization of comments
### Extracting the keywords from the list of offensive words text files

In [120]:
# Hurtful comments
def category1():

    keywords = set()
    with open("Hurtful.txt", encoding="utf-8") as list_file1:
        for line in list_file1:
            if line.strip():
                keywords.add(line.strip())
    
    with open("COMMENTSA.csv", encoding="utf-8") as master_file:
        with open("HurtfulCom.txt", 'w', encoding="utf-8") as search_results1:
            for line in master_file:
                if set(line.split()[:-1]) & keywords:
                    search_results1.write(' 1' + '\t' + line)  # Category 1
category1()

In [121]:
# Obscene comments
def category2():  
    
    keywords = set()
    with open("Obscene.txt", encoding="utf-8") as list_file2:
        for line in list_file2:
            if line.strip():
                keywords.add(line.strip())
    
    with open("COMMENTSA.csv", encoding="utf-8") as master_file:
        with open("ObsceneCom.txt", 'w', encoding="utf-8") as search_results2:
            for line in master_file:
                if set(line.split()[:-1]) & keywords:
                    search_results2.write(' 2' + '\t' + line)  # Category 2
category2() 

In [122]:
# Insulting comments
def category3():  
    
    keywords = set()
    with open("Insults.txt", encoding="utf-8") as list_file3:
        for line in list_file3:
            if line.strip():
                keywords.add(line.strip())
    
    with open("COMMENTSA.csv", encoding="utf-8") as master_file:
        with open("InsultCom.txt", 'w', encoding="utf-8") as search_results3:
            for line in master_file:
                if set(line.split()[:-1]) & keywords:
                    search_results3.write(' 3' + '\t' + line)  # Category 3
category3()   

In [123]:
# Racist comments
def category4():  
    
    keywords = set()
    with open("Racist.txt", encoding='latin1') as list_file4:
        for line in list_file4:
            if line.strip():
                keywords.add(line.strip())
    
    with open("COMMENTSA.csv", encoding="utf-8") as master_file:
        with open("RacistCom.txt", 'w', encoding="utf-8") as search_results4:
            for line in master_file:
                if set(line.split()[:-1]) & keywords:
                    search_results4.write(' 4' + '\t' + line)  # Category 4
category4()                   

In [124]:
# Combining all the files

filenames = ['HurtfulCom.txt', 'ObsceneCom.txt', 'InsultCom.txt', 'RacistCom.txt']
with open('BullyingCom.txt', 'w', encoding='utf=8') as outfile:
    for fname in filenames:
        with open(fname,encoding="utf-8") as infile:
            for line in infile:
                outfile.write(line)

In [125]:
documents2 = pd.read_csv('BullyingCom.txt',delimiter='\t',header=None)
documents2.rename(columns={0: 'Category',1:'Comments'}, inplace=True)
documents2.head()

,Category,Comments
0,1,ugly inside n outside sense die one shows s...
1,1,really yaaar u great patient sunny frustrated...
2,1,hypocrates everywhere ppl might curse fact pp...
3,1,asking questions giving chance speak hate typ...
4,1,sunny cool dumb questions hats u ur patience


## 4. MODEL BUILDING AND TESTING

In [137]:
def features(words):
    temp = str(word_tokenize(words))
    print(temp)

    words = [temp[0]]
    for i in range(1, len(temp)):
        if(temp[i] != temp[i-1]):
            words.append(temp[i])

    scoreF = BigramAssocMeasures.chi_sq
    bigrams = BCF.from_words(words).nbest(scoreF, 150)

    return dict([word,True] for word in itertools.chain(words, bigrams))

In [ ]:
'''
pos_sen = open("positive.txt", 'r', encoding = 'utf-8').read()
neg_sen = open("negative.txt", 'r', encoding = 'utf-8').read()

prev = [(features(words), 'positive') for words in pos_sen.split('\n')]
nrev = [(features(words), 'negative') for words in neg_sen.split('\n')]

ncutoff = int(len(nrev)*3/4)
pcutoff = int(len(prev)*3/4)
train_set = nrev[:ncutoff] + prev[:pcutoff] 
test_set = nrev[ncutoff:] + prev[pcutoff:]

#test_classifier = NaiveBayesClassifier.train(train_set)

test_classifier = DecisionTreeClassifier.train(train_set)


#SAVE IN FILE TO AVOID TRAIINING THE DATA AGAIN
save_doc = open("classifier.pickle", 'wb')
pickle.dump(test_classifier, save_doc)
save_doc.close()

'''

In [139]:
# Open the classifier pickle file

fl = open('classifier.pickle','rb')
classifier = pickle.load(fl)
fl.close()

In [140]:
print ("Accuracy is : ", util.accuracy(test_classifier, test_set) * 100)

Accuracy is :  83.78378378378379


## Comments classification model

In [141]:
feat1 = []
feat2 = []
feat3 = []
feat4 = []

f1,f2,f3,f4=0,0,0,0

with open('BullyingCom.txt', 'r', encoding="utf=8") as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        all_feats = dict(word_feats(row[0].split()).items())
        if int(row[0]) == 1:
            f1 += 1
            feat1.append((all_feats, 'Hurtful'))
        if int(row[0]) == 2:
            f2 += 1
            feat2.append((all_feats, 'Obscene'))
        if int(row[0]) == 3:
            f3+= 1
            feat3.append((all_feats, 'Insulting'))
        if int(row[0]) == 4:
            f4+= 1
            feat4.append((all_feats, 'Racist'))
print ("Hurtful comments count: ", f1)
print ("Obscene comments count: ",f2)
print ("Insulting comments count: ",f3)
print("Racist comments count: ",f4)

train = feat1 + feat2 + feat3 + feat4


Hurtful comments count:  462
Obscene comments count:  1658
Insulting comments count:  949
Racist comments count:  20


## Visualization

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

In [ ]:
objects = ('HURTFUL', 'OBSCENE', 'INSULTS', 'RACIST')
y_pos = np.arange(len(objects))
performance = [f1,f2,f3,f4]
plt.bar(y_pos, performance,align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Usage')
plt.title('Comment categorization')
plt.show()

In [142]:
accuracies = []
for x in range(10):
    random.shuffle(train)
    cutoff = int(len(train) * 0.01)
    trainfeats = train[:cutoff]
    testfeats = train[cutoff:]
 
    #classifier = NaiveBayesClassifier.train(trainfeats)
    classifier = DecisionTreeClassifier.train(trainfeats)
    accuracy = nltk.classify.util.accuracy(classifier, testfeats)
    accuracies.append(accuracy)
    print ((accuracy * 100))
print("These are the accuracies for our comment categorization classifier.")

    

99.3461915658712
99.3461915658712
99.3461915658712
99.3461915658712
99.3461915658712
99.3461915658712
99.3461915658712
100.0
100.0
99.3461915658712
These are the accuracies for our comment categorization classifier.


## User comment classification

In [143]:
usercomment=input("Enter a comment: ")

Enter a comment: Hey! What's up,doc? That person is a complete Idiot!😛 😜😝


### Data preprocessing

In [144]:
# emoji removal
with open("USERCOMMENT.csv", 'w', encoding="utf-8") as fp:
    emojiremoval(fp,usercomment) 

with open("USERCOMMENT.csv", 'r', encoding="utf-8") as fp:
    uline=fp.readlines()
uline      

["Hey! What's up,doc? That person is a complete Idiot! \n"]

In [145]:
# Text cleaning

with open("USERCOMMENT.csv", 'w+', encoding="utf-8") as fp:
    normalizing(fp,str(uline)) 

with open("USERCOMMENT.csv", 'r', encoding="utf-8") as fp:
    uline=fp.readlines()
uline

['  hey   doc  person complete idiot  n  \n']

In [146]:
# Sentiment analysis

with open("USERCOMMENT.csv", 'w', encoding="utf-8") as fp:
    sentiment(fp,str(uline))

with open("USERCOMMENT.csv", 'r', encoding="utf-8") as fp:
    uline=fp.readlines()
    
uline

["['  hey   doc  person complete idiot  n  \\n']|-0.35000000000000003\n"]

In [147]:
# Check if comment entered has negative or postive sentiment

for word in uline:
    if '-' in word:
        print("negative comment")
    else:
        print("positive comment")

negative comment


In [148]:
# Consider the words of length greater than 3 as tokens

tokenlist=[]
for i in uline:
    tokens = word_tokenize(i)
    print(tokens)
for i in tokens:
    if len(i)>=3 and i.isalpha():
        tokenlist.append(i)
tokenlist


['[', "'", 'hey', 'doc', 'person', 'complete', 'idiot', 'n', '\\n', "'", ']', '|-0.35000000000000003']


['hey', 'doc', 'person', 'complete', 'idiot']

In [149]:
# create a list of badwords

i=0
badwords=[]
filenames = ['Hurtful.txt', 'Obscene.txt', 'Insults.txt', 'Racist.txt']
while i!=4:
    with open(filenames[i]) as infile:
        for line in infile:
            badwords.append(line)
    i+=1

In [150]:
# The bullying or non-bullying comment classifier

flag=0
for t in tokenlist:
    for i in badwords:
        if t in i:
            flag=1
       
if flag==1:
    print("Your comment contains bullying words!")
else:
    print("Your comment is clean and ready to be posted!")

Your comment contains bullying words!
